## vLLM

## 初始環境設定

In [12]:
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path
current_foldr=!pwd
current_foldr=current_foldr[0]
current_foldr

'/work/g00cjz00/github/Llama2-Chinese/inference-speed/GPU/vllm_example'

## 安裝套件
安裝完成後建議, 點選上方選單, 直接階段->重新啟動工作階段, 確保 library重置

In [13]:
!pip install https://github.com/vllm-project/vllm/releases/download/v0.3.0/vllm-0.3.0+cu118-cp310-cp310-manylinux1_x86_64.whl -q
!pip install --upgrade xformers==0.0.23.post1 torch==2.1.2 --index-url https://download.pytorch.org/whl/cu118 -q

## 確認CUDA版本, 以及否能使用GPU
若無gpu 請點選右側->已連線->變更執行階段類型->T4 Gpu

In [14]:
!nvidia-smi
import torch
print(torch.__version__)  #注意是双下划线
torch.cuda.is_available()

Sun Feb 11 17:06:07 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:1B:00.0 Off |                    0 |
| N/A   32C    P0    41W / 300W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:1C:00.0 Off |                    0 |
| N/A   

True

## MEMO

In [ ]:
import sys
#sys.executable="/usr/bin/python3.10"
#!{sys.executable} -V

#from pprint import pprint
#pprint(sys.path)
#sys.path.append(’需要引用模块的地址')


###　Model Vllm

In [ ]:
!python3 -m vllm.entrypoints.api_server \
    --model /work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf --swap-space 16 \
    --disable-log-requests --host 0.0.0.0 --port 8080 --max-num-seqs 256

##　Model Openai

In [ ]:
!python3 -m vllm.entrypoints.openai.api_server \
--model /work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf --swap-space 16 \
--disable-log-requests --host 0.0.0.0 --port 5001 --max-num-seqs 20 --served-model-name llama-2

## CURL

In [ ]:
!curl http://gpn3001:5001/v1/models

In [ ]:
%%bash
curl http://gpn3001:5001/v1/completions \
-H "Content-Type: application/json" \
-d '{
"model": "llama-2",
"prompt": "San Francisco is a",
"max_tokens": 70,
"temperature": 0
}'

In [ ]:
%%bash
curl http://gpn3001:5001/v1/chat/completions \
-H "Content-Type: application/json" \
-d '{
"model": "llama-2",
"messages": [
{"role": "system", "content": "You are a helpful assistant."},
{"role": "user", "content": "什麼是聯邦式學習?"}
]
}'

## Singularity　Model Openai
change kernel to Pytohn 3 (ipykernel)

In [ ]:
!/work/opt/ohpc/Taiwania3/libs/singularity/3.10.2/bin/singularity exec --nv -C \
-B /work -B /work/u00cjz00/os/ubuntu/bin:/usr/ubuntu_bin \
-B /work/g00cjz00/libraryFolder/P-3.10-work-genai_cuda_11.8.0-cudnn8-runtime-ubuntu22.04/local/lib:/home/g00cjz00/.local/lib \
-B /work/g00cjz00/libraryFolder/P-3.10-work-genai_cuda_11.8.0-cudnn8-runtime-ubuntu22.04/local/bin:/home/g00cjz00/.local/bin \
/work/u00cjz00/nvidia/cuda_11.8.0-cudnn8-runtime-ubuntu22.04.sif \
bash -c 'export PATH=$PATH:$HOME/.local/bin; \
python3 -m vllm.entrypoints.openai.api_server \
--model /work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf --swap-space 16 \
--disable-log-requests --host 0.0.0.0 --port 5001 --max-num-seqs 20 --served-model-name llama-2'

INFO 02-11 17:12:12 api_server.py:209] args: Namespace(host='0.0.0.0', port=5001, allow_credentials=False, allowed_origins=['*'], allowed_methods=['*'], allowed_headers=['*'], api_key=None, served_model_name='llama-2', chat_template=None, response_role='assistant', ssl_keyfile=None, ssl_certfile=None, root_path=None, middleware=[], model='/work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf', tokenizer=None, revision=None, tokenizer_revision=None, tokenizer_mode='auto', trust_remote_code=False, download_dir=None, load_format='auto', dtype='auto', kv_cache_dtype='auto', max_model_len=None, worker_use_ray=False, pipeline_parallel_size=1, tensor_parallel_size=1, max_parallel_loading_workers=None, block_size=16, seed=0, swap_space=16, gpu_memory_utilization=0.9, max_num_batched_tokens=None, max_num_seqs=20, max_paddings=256, disable_log_stats=False, quantization=None, enforce_eager=False, max_context_len_to_capture=8192, disable_custom_all_reduce=False, enable_lora=False, max_loras=1